In [1]:
import pandas as pd
from sqlalchemy import create_engine
import snowflake.connector
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables from .env file
load_dotenv()

# Retrieve Snowflake credentials from environment variables
user = os.getenv("SNOWFLAKE_USER")
password = os.getenv("SNOWFLAKE_PASSWORD")
account = os.getenv("SNOWFLAKE_ACCOUNT")
warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
database = os.getenv("SNOWFLAKE_DATABASE")
schema = os.getenv("SNOWFLAKE_SCHEMA")

In [3]:
# Connect to Snowflake
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

# Create a cursor
cur = conn.cursor()

# Test connection
cur.execute("SELECT CURRENT_VERSION()")
print("Connected to Snowflake - Version:", cur.fetchone()[0])

Connected to Snowflake - Version: 9.4.3


In [4]:
# Load environment variables
load_dotenv()

# Retrieve Snowflake credentials from environment variables
user = os.getenv("SNOWFLAKE_USER")
password = os.getenv("SNOWFLAKE_PASSWORD")
account = os.getenv("SNOWFLAKE_ACCOUNT")
warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
database = os.getenv("SNOWFLAKE_DATABASE")
schema = os.getenv("SNOWFLAKE_SCHEMA")

# URL encode password to handle special characters
import urllib.parse
encoded_password = urllib.parse.quote_plus(password)

print(encoded_password)

Sohammane%402001


In [5]:
import urllib.parse
encoded_password = urllib.parse.quote_plus(password)

# Create the Snowflake SQLAlchemy URL with the encoded password
snowflake_url = f"snowflake://{user}:{encoded_password}@{account}/{database}/{schema}?warehouse={warehouse}"

# Create the engine using the Snowflake connection URL
engine = create_engine(snowflake_url)


# Query to load your data into a pandas DataFrame
query = "SELECT * FROM real_estate_data"
df12 = pd.read_sql(query, engine)

# Check if data is loaded correctly
print(df12.head())

   total_sqft  bath  price  bhk  1st_Block_Jayanagar  1st_Phase_JP_Nagar  \
0      2850.0   4.0  428.0    4                 True               False   
1      1630.0   3.0  194.0    3                 True               False   
2      1875.0   2.0  235.0    3                 True               False   
3      1200.0   2.0  130.0    3                 True               False   
4      1235.0   2.0  148.0    2                 True               False   

   2nd_Phase_Judicial_Layout  2nd_Stage_Nagarbhavi  5th_Block_Hbr_Layout  \
0                      False                 False                 False   
1                      False                 False                 False   
2                      False                 False                 False   
3                      False                 False                 False   
4                      False                 False                 False   

   5th_Phase_JP_Nagar  ...  vijayanagar  vishveshwarya_layout  \
0               False

**Model Building**

In [6]:
df12.shape

(7239, 244)

In [7]:
X = df12.drop(['price'],axis='columns')
X.head(3)

,total_sqft,bath,bhk,1st_Block_Jayanagar,1st_Phase_JP_Nagar,2nd_Phase_Judicial_Layout,2nd_Stage_Nagarbhavi,5th_Block_Hbr_Layout,5th_Phase_JP_Nagar,6th_Phase_JP_Nagar,...,vijayanagar,vishveshwarya_layout,vishwapriya_layout,vittasandra,whitefield,yelachenahalli,yelahanka,yelahanka_new_town,yelenahalli,yeshwanthpur
0,2850.0,4.0,4,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1630.0,3.0,3,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1875.0,2.0,3,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
X.shape

(7239, 243)

In [9]:
y = df12.price
y.head(3)

0    428.0
1    194.0
2    235.0
Name: price, dtype: float64

In [10]:
len(y)

7239

In [11]:
from sklearn.model_selection import train_test_split  # type: ignore
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [12]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

0.8629132245229443

**Use K Fold cross validation to measure accuracy of our LinearRegression model**

In [13]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

array([0.82702546, 0.86027005, 0.85322178, 0.8436466 , 0.85481502])

**We can see that in 5 iterations we get a score above 80% all the time. This is pretty good but we can test few other algorithms for regression to see if we can get even better score. In the end we can use GridSearchCV for this purpose**

Find best model using GridSearchCV

In [14]:
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor
import pandas as pd

def find_best_model_using_gridsearchcv(X, y):
    algos = {
        'linear_regression': {
            'model': LinearRegression(),
            'params': {
                'fit_intercept': [True, False],
                'positive': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1, 2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['squared_error', 'friedman_mse'],
                'splitter': ['best', 'random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X, y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

find_best_model_using_gridsearchcv(X, y)


,model,best_score,best_params
0,linear_regression,0.847951,"{'fit_intercept': False, 'positive': False}"
1,lasso,0.726738,"{'alpha': 2, 'selection': 'cyclic'}"
2,decision_tree,0.717489,"{'criterion': 'friedman_mse', 'splitter': 'ran..."


In [15]:
import numpy as np
def predprice(location, sqft, bath, bhk):
    try:
        loc_index = np.where(X.columns == location)[0][0]
    except IndexError:
        print(f"Warning: Location '{location}' not found in training data.")
        return None  # Return None if location is not in X.columns

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return float(lr_clf.predict([x])[0])

In [16]:
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

In [21]:
predprice('indira_nagar', 1000, 3, 2)

197.02892403856663

In [23]:
predprice('2nd_Stage_Nagarbhavi', 1000, 3, 2)

187.92065535874656

In [24]:
import pickle
with open('price_prediction.pickle','wb') as f:
    pickle.dump(lr_clf,f)

In [25]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))